# Drawdowns

It's important to note that drawdown is NOT a perfect measure!

First, drawdowns are entirely defined by two points and hence are very sensitive to outliers.

Second, they depend on the frequency of observations in general. 

In spite of these shortcomings, drawdowns are closely monitored and popular amongst practitioners even though other measures like VAR and CVAR are more robust measures of extreme risk.

In [67]:
import pandas as pd

In [68]:
me_m = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv", header=0, index_col=0, parse_dates=True, na_values=-99.99)
rets = me_m[['Lo 10', 'Hi 10']]
rets.columns = ['SmallCap', 'LargeCap']
rets = rets/100
#rets.plot.line()

In [69]:
rets.index

Int64Index([192607, 192608, 192609, 192610, 192611, 192612, 192701, 192702,
            192703, 192704,
            ...
            201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810,
            201811, 201812],
           dtype='int64', length=1110)

In [70]:
rets.index = pd.to_datetime(rets.index, format="%Y%m")

In [71]:
#rets.plot.line()

In [72]:
rets.head()

,SmallCap,LargeCap
1926-07-01,-0.0145,0.0329
1926-08-01,0.0512,0.0370
1926-09-01,0.0093,0.0067
1926-10-01,-0.0484,-0.0243
1926-11-01,-0.0078,0.0270


# Compute Drawdowns

1. Compute a wealth index
2. Compute previous peaks
3. Compute drawdown - which is the wealth value - as a percentage of the previous peak 

In [73]:
wealth_index = 1000*(1+rets['LargeCap']).cumprod()

In [74]:
wealth_index.head()

1926-07-01    1032.900000
1926-08-01    1071.117300
1926-09-01    1078.293786
1926-10-01    1052.091247
1926-11-01    1080.497711
Name: LargeCap, dtype: float64

In [75]:
#wealth_index.plot.line()

In [76]:
previous_peaks = wealth_index.cummax()

In [77]:
#previous_peaks.plot.line()

In [78]:
drawdown = (wealth_index - previous_peaks)/previous_peaks
#drawdown.plot()

In [79]:
drawdown.head()

1926-07-01    0.0000
1926-08-01    0.0000
1926-09-01    0.0000
1926-10-01   -0.0243
1926-11-01    0.0000
Name: LargeCap, dtype: float64

In [80]:
drawdown.min() #Maximum Drawdown

-0.8400375277943123

In [81]:
drawdown["1975":].idxmin()

Timestamp('2009-02-01 00:00:00')

In [82]:
drawdown.idxmin()

Timestamp('1932-05-01 00:00:00')

# Create a drawdown function for reproduction

In [83]:
def drawdown(return_series: pd.Series):
    """
    Takes a time series of asset returns
    Computes and returns a DataFrame that contains:
    the wealth index
    the previous peaks
    percent drawdowns
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, "Peaks": previous_peaks, "Drawdown": drawdowns})

In [84]:
drawdown(rets['LargeCap']).head()

,Wealth,Peaks,Drawdown
1926-07-01,1032.900000,1032.900000,0.0000
1926-08-01,1071.117300,1071.117300,0.0000
1926-09-01,1078.293786,1078.293786,0.0000
1926-10-01,1052.091247,1078.293786,-0.0243
1926-11-01,1080.497711,1080.497711,0.0000


In [85]:
drawdown(rets['LargeCap'])[["Wealth", "Peaks"]].head()

,Wealth,Peaks
1926-07-01,1032.900000,1032.900000
1926-08-01,1071.117300,1071.117300
1926-09-01,1078.293786,1078.293786
1926-10-01,1052.091247,1078.293786
1926-11-01,1080.497711,1080.497711


In [86]:
#drawdown(rets['LargeCap'])[["Wealth", "Peaks"]].plot()

In [87]:
#drawdown(rets[:"1950"]['LargeCap'])[["Wealth", "Peaks"]].plot()

In [88]:
drawdown(rets['LargeCap'])["Drawdown"].min()

-0.8400375277943123

In [89]:
drawdown(rets['SmallCap'])["Drawdown"].min()

-0.8330007793945303

In [90]:
drawdown(rets['LargeCap'])["Drawdown"].idxmin()

Timestamp('1932-05-01 00:00:00')

In [91]:
drawdown(rets['SmallCap'])["Drawdown"].idxmin()

Timestamp('1932-05-01 00:00:00')

In [92]:
drawdown(rets["1940":]['SmallCap'])["Drawdown"].min()

-0.7000596091967917

In [93]:
drawdown(rets["1940":]['SmallCap'])["Drawdown"].idxmin()

Timestamp('1974-12-01 00:00:00')